<a href="https://colab.research.google.com/github/atick-faisal/Hand-Gesture-Recognition/blob/main/Classical-ML-Analysis/Static_Hand_Gestures_ML_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import time
import joblib
import shutil
import tarfile
import requests

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from scipy.signal import butter, lfilter

from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.utils import to_categorical

from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

from tensorflow import feature_column
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical

In [3]:
DATASET_ID      = '1p0CSRb9gax0sKqdyzOYVt-BXvZ4GtrBv'

# -------------BASE DIR (MODIFY THIS TO YOUR NEED) ------------ #
# BASE_DIR        = '../'
BASE_DIR     = '/content/drive/MyDrive/Research/Hand Gesture/GitHub/'

DATA_DIR        = 'Sensor-Data/'
CHANNELS_DIR    = 'Channels/'
FIGURE_DIR      = 'Figures/'
LOG_DIR         = 'Logs/'

USERS           = ['001', '002', '003', '004', '005', '006', '007', '008', '009',
                   '010', '011', '012', '013', '014', '015', '016', '017', '018',
                   '019', '020', '021', '022', '023', '024', '025']

GESTURES        = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 
                   'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y']

EPOCHS          = 10
BATCH_SIZE      = 512

CHANNELS_GROUP  = 'STATIC_GESTURES_ONLY_'
CUT_OFF         = 3.0
ORDER           = 4
FS              = 100

CONFIG          = "Rolling median filter for flex, LPF for IMU, RnF\n"

In [4]:
#--------------------- Download util for Google Drive ------------------- #

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)
        
    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value
        
    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk:
                f.write(chunk)

def download_data(fid, destination):
    print('cleaning already existing files ... ', end='')
    try:
        shutil.rmtree(destination)
        print('√')
    except:
        print('✕')
        
    print('creating data directory ... ', end='')
    os.mkdir(destination)
    print('√')
    
    print('downloading dataset from the repository ... ', end='')
    filename = os.path.join(destination, 'dataset.tar.xz')
    try:
        download_file_from_google_drive(fid, filename)
        print('√')
    except:
        print('✕')
        
    print('extracting the dataset ... ', end='')
    try:
        tar = tarfile.open(filename)
        tar.extractall(destination)
        tar.close()
        print('√')
    except:
        print('✕')

In [5]:
# ------- Comment This if already downloaded -------- #

# destination = os.path.join(BASE_DIR, DATA_DIR)
# download_data(DATASET_ID, destination)

In [6]:
class LowPassFilter(object): 
    def butter_lowpass(cutoff, fs, order):
        nyq = 0.5 * fs
        normal_cutoff = cutoff / nyq
        b, a = butter(order, normal_cutoff, btype='low', analog=False)
        return b, a

    def apply(data, cutoff=CUT_OFF, fs=FS, order=ORDER):
        b, a = LowPassFilter.butter_lowpass(cutoff, fs, order=order)
        y = lfilter(b, a, data)
        return y

In [7]:
def clean_dir(path):
    print('cleaning already existing files ... ', end='')
    try:
        shutil.rmtree(path)
        print('√')
    except:
        print('✕')
    
    print('creating ' + path + ' directory ... ', end='')
    os.mkdir(path)
    print('√')

def extract_channels():
    channels_dir = os.path.join(BASE_DIR, CHANNELS_DIR)
    clean_dir(channels_dir)
        
    for user in USERS:
        print('Processing data for user ' + user, end=' ')
        
        features = pd.DataFrame()
        
        for gesture in GESTURES:
              
            user_dir = os.path.join(BASE_DIR, DATA_DIR, user)
            gesture_dir = os.path.join(user_dir, gesture + '.csv')

            dataset = pd.read_csv(gesture_dir)

            dataset['flex_1'] = dataset['flex_1'].rolling(3).median()
            dataset['flex_2'] = dataset['flex_2'].rolling(3).median()
            dataset['flex_3'] = dataset['flex_3'].rolling(3).median()
            dataset['flex_4'] = dataset['flex_4'].rolling(3).median()
            dataset['flex_5'] = dataset['flex_5'].rolling(3).median()

            dataset.fillna(0, inplace=True)
            
            flx1 = dataset['flex_1'].to_numpy()
            flx2 = dataset['flex_2'].to_numpy()
            flx3 = dataset['flex_3'].to_numpy()
            flx4 = dataset['flex_4'].to_numpy()
            flx5 = dataset['flex_5'].to_numpy()
            
            accx = dataset['ACCx'].to_numpy()
            accy = dataset['ACCy'].to_numpy()
            accz = dataset['ACCz'].to_numpy()
            
            accx = LowPassFilter.apply(accx)
            accy = LowPassFilter.apply(accy)
            accz = LowPassFilter.apply(accz)
            
            gyrx = dataset['GYRx'].to_numpy()
            gyry = dataset['GYRy'].to_numpy()
            gyrz = dataset['GYRz'].to_numpy()
            
            gyrx = LowPassFilter.apply(gyrx)
            gyry = LowPassFilter.apply(gyry)
            gyrz = LowPassFilter.apply(gyrz)
            
            accm = np.sqrt(accx ** 2 + accy ** 2 + accz ** 2)
            gyrm = np.sqrt(gyrx ** 2 + gyry ** 2 + gyrz ** 2)
            
            g_idx = GESTURES.index(gesture)
            labels = np.ones((accx.shape[0], )) * g_idx
            
            channels = np.stack([
                flx1, flx2, flx3, flx4, flx5,
                accx, accy, accz, labels
            ], axis=-1)

            _features = pd.DataFrame(channels)
            features = pd.concat([features, _features], ignore_index=True)
            
        
        features.columns = [
                'flx1', 'flx2', 'flx3', 'flx4', 'flx5',
                'accx', 'accy', 'accz', 'labels'
        ]
        path = os.path.join(BASE_DIR, CHANNELS_DIR, CHANNELS_GROUP + user + '_features.joblib')
        joblib.dump(features, path)
        
        print('√')

In [8]:
# extract_channels()

In [9]:
def df_to_dataset(dataframe, shuffle=True, batch_size=BATCH_SIZE):
    dataframe = dataframe.copy()
    labels = dataframe.pop('labels')
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [10]:
def get_model():
    inputs = {}
    feature_columns = []
    headers = ['flx1', 'flx2', 'flx3', 'flx4', 'flx5', 'accx', 'accy', 'accz']
    
    for header in headers:
        inputs[header] = tf.keras.Input(shape=(1,), name=header) 
        feature_columns.append(feature_column.numeric_column(header))

    x = tf.keras.layers.DenseFeatures(feature_columns)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(.5)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(.1)(x)
    x = layers.Dense(len(GESTURES))(x)

    model = tf.keras.models.Model(inputs=inputs, outputs=x)
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)

    model.compile(
        optimizer=opt,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [ ]:
ACC = []
logs = ''

for test_user in USERS:
    print('Processing results for user ' + test_user, end='... \n')
    
    train = pd.DataFrame()
    test = pd.DataFrame()

    for user in USERS:
        feature_path = os.path.join(BASE_DIR, CHANNELS_DIR, CHANNELS_GROUP + user + '_features.joblib')
        features = joblib.load(feature_path)

        if user == test_user:
            test = pd.concat([test, features], ignore_index=True)
        else:
            train = pd.concat([train, features], ignore_index=True)

    train_ds = df_to_dataset(train)
    test_ds = df_to_dataset(test)

    model = get_model()
    model.fit(train_ds, epochs=EPOCHS)

    _, accuracy = model.evaluate(test_ds)

    accuracy = accuracy * 100
    print(f'%.2f %%' %(accuracy))
    logs = logs + 'Accuracy for user ' + str(test_user) + '... ' + str(accuracy) + '\n'
    ACC.append(accuracy)
    
AVG_ACC = np.mean(ACC)
STD = np.std(ACC)
print('------------------------------------')
print(f'Average accuracy %.2f +/- %.2f' %(AVG_ACC, STD))

Processing results for user 001... 
Epoch 1/10
1688/1688 [==============================] - 15s 7ms/step - loss: 2.6572 - accuracy: 0.3337
Epoch 2/10
1688/1688 [==============================] - 13s 7ms/step - loss: 1.3764 - accuracy: 0.5883
Epoch 3/10
1688/1688 [==============================] - 13s 7ms/step - loss: 1.0785 - accuracy: 0.6694
Epoch 4/10
1688/1688 [==============================] - 13s 7ms/step - loss: 0.9485 - accuracy: 0.7090
Epoch 5/10
1688/1688 [==============================] - 13s 7ms/step - loss: 0.8762 - accuracy: 0.7377
Epoch 6/10
1688/1688 [==============================] - 14s 7ms/step - loss: 0.7918 - accuracy: 0.7564
Epoch 7/10
1688/1688 [==============================] - 14s 7ms/step - loss: 0.7149 - accuracy: 0.7729
Epoch 8/10
1688/1688 [==============================] - 13s 7ms/step - loss: 0.6568 - accuracy: 0.7856
Epoch 9/10
1688/1688 [==============================] - 13s 7ms/step - loss: 0.6436 - accuracy: 0.7963
Epoch 10/10
71/71 [==================